This notebook is the one which analyses the videos. It takes the videos and provides a text file containing matices for the transitions and transition times for all the videos.

This version is good for when there is a majority of one particle type. When the two are even, there has to be an account fro mixed bonding so the code's a little different. Remember though to set all the variables in the final block of code according to the actual data you took. These are the 'bond lenghts', and they are inferred from a sample of videos using the 'shape testing' code. 

In [1]:
"""Imports"""
import matplotlib.pyplot as plt #for maknig plots inside the notebook
import skimage
import skvideo.io
from skvideo.io import vreader, ffprobe
from skimage import measure, morphology, feature
from skimage.filters import *
from skimage.morphology import *
from operator import attrgetter
import numpy as np
import math
from glob import glob
import seaborn as sns
from collections import OrderedDict 

blob_doh = skimage.feature.blob_doh
blob_dog = skimage.feature.blob_dog

This code block processes single frames. It is written for the 3-styrene ('glas'), 1-polyethylene ('plas') case. To adapt this code for another case, like having three ethylene, replace 'glas on glas' (styrene-styrene bonding) with 'plas on plas', and swap out 'plas' and 'glas' for each other everywhere. 'glas' is, in this code, whichever is the most frequent type, and 'plas' is the material of which there is only one particle.

In [2]:
"""Functions"""
# This does most of the work. 
# It takes in a single frame and classifies the raft of particles in it as one of the epected shapes 
# or as unclassified, which here includes broken rafts (particles which are not in a complete raft of 4)
# and also mis-shapen rafts, such as rafts with a gap in the middle if it'sbig enough, and rafts that are ambiguous to the program
def total_threshold_filter(frame, frame_no, transition_threshold, class_thresh,
                           broken_count, class_num, origin, last_class, last_whole, state_start, filtrate_len, 
                           classes, transitions, times): 
    
   
   # ---detect blobs ----------------
    thresh_img = frame > threshold_isodata(frame)# binary image
        
    # The ethylene blobs
    phighlight = binary_opening(thresh_img, square(14))
    img = np.copy(frame)
    img[phighlight==0] = 0
    pblobs =blob_doh(img, min_sigma = 10, max_sigma = 16, threshold = 0.007, num_sigma= 15, overlap=0.6)
    
    # The styrene blobs
    ghighlight = opening(
        white_tophat(gaussian(thresh_img, sigma=0.2), square(14)),
                                   disk(4))
    img = np.copy(frame)
    img[ghighlight==0] = 0
    gblobs =blob_dog(img, min_sigma = 8, max_sigma = 12, threshold = 0.2, overlap=0.5)
    gblobs[:, 2] = gblobs[:, 2] * np.sqrt(2)

    # The number of each blob
    ngblobs = len(gblobs)
    npblobs = len(pblobs) 
    num_blobs = ngblobs + npblobs
    
    # These two collect the number of bonds there are, between ethylene balls and betwee ethylene and styrene balls
    plas_on_glas_dists =[connect((pblob[1], pblob[0]),m_bond, gblobs) for pblob in pblobs] #mixed bonding
    num_mconnections = reducer(plas_on_glas_dists)


    glas_on_glas_dists =[connect((gblobs[i][1], gblobs[i][0]),g_bond, gblobs[i+1:]) for i in range(ngblobs-1)] # same type bonding
    num_gconnections = reducer(glas_on_glas_dists)



    num_connections = sum([num_gconnections,num_mconnections])
    
    # keep frames that have enough paricles, with enough connectivity to be a bonded raft
    #        Sometimes the junk reflections from the transducer make extra blobs, hence the '>'
    test = (num_connections >= expected_connections and num_blobs>=expected_blobs)
    
    if test:
        clas =sideify(num_gconnections, num_mconnections ) 
        classes[clas]+=1  # count 1 instance of this shape
        
        if clas == 'ucf': # unclassifiable
            pass
        else:
            if origin == 'ucf': #saety in case video starts on a broken frame
                origin = last_class
            if clas == last_class: # if the raft has kept its shape since last frame
                class_num += 1
            else:
                class_num = 0 # reset the stability counter
                
            # If the raft has been stable for longer than the stability threshold in its current state
            if class_num >= class_thresh: 
                # If the raft was broken for longer than the broken threshold before attaining this state
                # or if current state is not the same as the previous stable state,
                # record a transition under the appropriate key in the dictionary
                # and record the three measures of transition time in their dictionary
                if broken_count >= transition_threshold or clas != origin:
                    transitions[origin + "->" + clas]+=1
                    
                    between_time = frame_no - state_start                               
                    times[origin + "->" + clas][0].append(between_time)  

                    stable_time = last_whole - state_start
                    times[origin + "->" + clas][1].append(stable_time)

                    unstable_time = frame_no - last_whole                      
                    times[origin + "->" + clas][2].append(unstable_time)
                
                    state_start = frame_no # the start of a new state, this state
                    
                    origin = clas # reset the 'previous state' label to this state
                    
                broken_count = 0 # since the raft is now stable, reset the time for which it has been broken
                last_whole = frame_no # and update the last recorded stable frame to this one
                
            filtrate_len += 1 # this is the number of frames which were classifiable, and it's not used afterwards
            last_class = clas
            
            
    else:
        class_num = 0 # if it's not a well-formed raft, reset the sability counter
        broken_count += 1 # and update the time for which it's been broken

    return [broken_count, class_num, origin, last_class, last_whole,state_start, filtrate_len]



        
# This is the shape classifier
def sideify( num_gconnections, num_mconnections):    
    if num_mconnections == 2 and num_gconnections ==3:# and num_mconnections==4: #gtotallengths>48
        return "t"
    elif num_mconnections == 3 and num_gconnections ==2:# and num_mconnections == 4:
        return "c"
    else:
        return "ucf"

# just a summer
def reducer(connections):
    return sum(connections) 

# given a focal point, a radiuus of connectiity, and a list of other circles
# determine which of a list of them it is connected to
def connect(cv, radius, bloblist): 
    num_connections = 0
    for blob2 in bloblist:
        cv2 = (blob2[1], blob2[0]) # center of blob2
        vector_d = vector_dist(cv, cv2)
        if vector_d <= 2*radius and vector_d > .2*radius: # blob centers closer than diameter of one blob
            num_connections += 1
        else:
            pass
    return (num_connections)

def vector_dist(v1, v2): # euclidean distance between 2 points
    return math.sqrt(np.sum([(v1[i] - v2[i])**2 for i in range(len(v1))]))

def third_item(l1):
    return l1[2]   

This code block applies the previous code to every frame, and every video. Intermediate results are printed. Typically, I copy the printed tet into a text file myself, and that is how the other programs I have written expect it to be presented. I do save the results to a file for safekeeping; however it is in a different format and I don't read from it any longer. You could modiy this code to write directly the displayed text to a save file in its current format, or modify the Mari Multiplication code to read from the saved file's set of arrays instead, for a change.

In [3]:
filenames = glob("./tchakamau/De*styrene/*[0-9].avi") # all the video data 
filenames.sort() # want to go through them in a reproducible order in case we have to restart halfway through
print(filenames)
num_top_keys = 3 # number of states plus 'unlclassified'
num_full_keys = 3
num_tran_keys = 2 # number of states
num_lengths = 3 # number of measures of transition time
trans_threshes = np.linspace(30, 60, 2) # Place the transition thesholds you would like to use here
class_threshes =[20,30] # and the stability thresholds you want; every combination of these is done
trans_runs = {} # this is the dictionary holding the results

# this is a list of parameters custom chosen for this data using 'shape testing'
ave = np.average
expected_blobs = 4 # number of particles to a raft
expected_connections = 5 # number of bonds in a raft
g_bond = 15*1 # glass bonding distance cutoff
m_bond = 15*1.15 # mixed bonding cutoff


params = [expected_blobs , expected_connections, g_bond, m_bond] # stores the key parameters in the save file to document the run

print("Number files: ", len(filenames))

# Let's get started
index = 0 # count the number of threshold pairs 
for trans_fil in trans_threshes:     
    for class_thresh in class_threshes:
        
        Full_classif = np.zeros((len(filenames), num_full_keys)) # empty array for the no. frames spent in each state
        N_transitions = np.zeros((len(filenames), num_tran_keys, num_tran_keys)) # and for the transitions in this video
        T_transitions = [[[[[] for i in range(num_tran_keys)] for j in range(num_tran_keys)] for k in range(num_lengths)] 
                          for l in range(len(filenames))] # and a list for the times
        num_total_frames = 0 
        filtrate_len = 0
        for vidnum in range(len(filenames)):
            print("Processing vid %s : %s" %(vidnum, filenames[vidnum]))
            frames = vreader(filenames[vidnum]) # does not load whole video
            # makes an object to load single frames at a time later

            # Every video collects the results into these dictionaries, and pass them onto the arrays when the video ends
            classes = [("t",0), ("c",0), ("ucf",0)]
            transitions = [("t->t",0), ("t->c",0),
                          ("c->t" ,0), ("c->c",0)]
            times = [("t->t",[[] for i in range(num_lengths)]), ("t->c",[[] for i in range(num_lengths)]),
                          ("c->t" ,[[] for i in range(num_lengths)]), ("c->c",[[] for i in range(num_lengths)])]
            transitions = OrderedDict(transitions)
            times = OrderedDict(times)            
            classes = OrderedDict(classes)
            
            # all these are updated frame by frame 
            broken_count = 0
            class_num = 0
            origin = 'ucf'
            last_class = 'ucf'
            last_whole = 0
            state_start = 0
            frame_no = 0
            
            # And we call every frame now
            for fr in frames:
                frame = fr[:, :, 2] # take only the green channel for greyscale
                frame_no += 1
                num_total_frames += 1
                # call our classifying function, and use it to update our parameters
                org = total_threshold_filter(frame, frame_no, trans_fil, class_thresh,
                                             broken_count, class_num, origin, last_class, last_whole, state_start, filtrate_len, 
                                              classes, transitions, times)
                broken_count, class_num, origin, last_class, last_whole, state_start, filtrate_len = org
            
            #-------Sort the results out from this video and add to the collector arrays/lists------------------
            ckeys = list(classes.keys())
            clcs = [classes[x] for x in ckeys] # list of frame nums per class
            
            tkeys = list(times.keys())
            tims = [times[x] for x in tkeys]
            
            trkeys = list(transitions.keys())
            trans = [transitions[x] for x in trkeys] # num of each transition

            
            Full_classif[vidnum,:] = clcs
            N_transitions[vidnum,:] = np.reshape(trans, (num_tran_keys, num_tran_keys))             
            T_transitions[vidnum] = np.moveaxis(np.reshape(tims, (num_tran_keys, num_tran_keys,num_lengths)),
                                                  [0,1,2], [1,2,0])
            
            # So we all follow along (and can collect results if it ends early)
            print("T threshold: %s" %trans_fil)
            print("C threshold: %s" %class_thresh)
            print(N_transitions[vidnum,:])
            print(T_transitions[vidnum])
            print(Full_classif[vidnum,:])
            
        trans_runs[index] = [[Full_classif, (N_transitions, T_transitions), (trans_fil, class_thresh), num_full_keys,
                                 num_tran_keys, ckeys, num_total_frames], params]
        index += 1
        #---------------------------------------------------------------------------------

#-----------------------------------------------------------------------

np.save('your_favourite_filename.npy',trans_runs)


[]
Number files:  0


NameError: name 'ckeys' is not defined